# 한글폰트 설치
- 아래 실행후 '런터임 다시 시작' 하면 됩니다.
- 메뉴에서 '런타임' >> '런타임 다시 시작'을 클릭하세요.
- 런타임 다시 시작 후 다시 실행할 필요 없습니다.

In [ ]:
# 폰트 설치
!apt-get update -qq # 나눔고딕 설치
!apt-get install fonts-nanum* -qq

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 로딩
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

# Install

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 9.7MB/s 


# Evn

In [ ]:
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# random seed initialize
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [ ]:
!nvidia-smi

Mon Jan 25 08:45:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Gensim
- https://radimrehurek.com/gensim/index.html

In [ ]:
import gensim
import gensim.downloader as api

## 한국어 학습

- https://wikidocs.net/50739

In [ ]:
# 행태소분석기 설치
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

In [ ]:
import konlpy
okt = konlpy.tag.Okt()

In [ ]:
okt.morphs("아버지가방에들어가신다")

In [ ]:
# 네이버 영화 리뷰 데이터 다운로드
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt

In [ ]:
nsmc_data = pd.read_csv("ratings.txt", header=0, delimiter='\t', quoting=3)
print(f"전체 데이터의 개수: {len(nsmc_data)}")
nsmc_data.head(10)

In [ ]:
# null 제거
nsmc_data = nsmc_data.dropna()
print(f"null 제거 후 데이터의 개수: {len(nsmc_data)}")
nsmc_data.head(10)

In [ ]:
# 한글 이외의 문자 제거
nsmc_data['document'] = nsmc_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
nsmc_data.dropna()
print(f"한글 아닌 문자 제거 후 데이터의 개수: {len(nsmc_data)}")
nsmc_data.head(10)

In [ ]:
# 불용어 정의 (빈도가 너무 많은 단어는 학습에서 제외 함)
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
# okt 형태소 분석기를 이용해 형태소 단위로 분할 (이때 불용어 제거)
tokens = []
for i, document in enumerate(tqdm(nsmc_data['document'], total=len(nsmc_data))):
    line = []
    line = okt.morphs(document)
    line = [word for word in line if not word in stopwords]
    tokens.append(line)

In [ ]:
print(len(tokens))
tokens[:10]

In [ ]:
# gensim 학습
word2vec_100 = gensim.models.Word2Vec(sentences=tokens, size=100, window=5, min_count=5)

In [ ]:
words = list(word2vec_100.wv.vocab)
len(words), words[:100]

In [ ]:
similar = word2vec_100.wv.most_similar("영화")
similar

In [ ]:
similar = word2vec_100.wv.most_similar("최민수")
similar

In [ ]:
similar = word2vec_100.wv.most_similar("장동건")
similar

In [ ]:
# 설경구 - 송윤아 + 고소영
result = word2vec_100.wv.most_similar(positive=['고소영', '설경구'], negative=['송윤아'])
result